# nlp-handson 自然言語処理ハンズオン

機械学習フレームワーク keras を用いて自然言語処理(分類)を行います。

自然言語分類は、ある文章を、用意したそれぞれの分類にどのくらい当てはまるかを返します。例えばチャットボットのように、ユーザーから得られる無限にある文章のパターンを分類して特定の答えを返すような事ができます。

- `main.ipynb` でtensorflow hubのuniversal-sentence-encoderを利用した転移学習で自然言語分類について
- `serving.ipynb` でtensorflow servingを利用したAPI化について
- `client.ipynb` でそのAPIの利用方法について

In [1]:
import csv
from random import shuffle

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tf_sentencepiece
from keras import backend as K
from keras.engine import Layer
import keras.layers as layers
import keras.optimizers as optimizers
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

W0605 23:15:12.304090 140692289308416 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


用意されたdatasets.csvを読み込み、分類をキーにサンプルデータを用意します。

__学習に時間がかかるためdatasets.csvは7分類各5個とかなりスモールデータな例で用意しています。__

__パラメータにもよりますが500分類各100個だとCPUでだいたい30分ほどかかります。さらに大きいデータではSagemakerなどのGPUを使った学習を検討しましょう__

また、今回は軽く試す程度に止めるため前処理を全くしていません。

[自然言語処理における前処理の種類とその威力](https://qiita.com/Hironsan/items/2466fe0f344115aff177)を参考に前処理を検討しましょう。

In [2]:
csv_path = './datasets.csv'
datasets = {}
with open(csv_path, 'r') as f:
    reader = csv.reader(f)
    for index, row in enumerate(reader):
        collected_row = [sentence for sentence in row if not sentence == '']
        if collected_row[0] in datasets:
            raise
        # 先頭をlabelとする
        datasets[collected_row[0]] = collected_row

In [3]:
datasets

{'ファッション': ['ファッション',
  '帽子買いたい',
  'ジーパン欲しい',
  'Tシャツ買いたい',
  '靴がボロくなった',
  'ジャケット買わないと'],
 'スーパー': ['スーパー', '夕食の食材買いたい', '野菜買わないと', '卵足りない', '牛乳飲みたい', '日用品がなくなってきた'],
 'レストラン': ['レストラン', '腹減った', '飯くいたい', 'ランチどこにしよう', 'ステーキが欲しい', 'お腹ぺこぺこ'],
 'コンビニ': ['コンビニ', 'コーヒー飲みたい', 'タバコ買いたい', 'ひと息つきたい', 'お菓子買いたい', '休憩しよう'],
 'トイレ': ['トイレ', '漏れそう', '用を足したい', 'お手洗いはどこですか', '化粧室はどこ', '手を洗いたい'],
 '病院': ['病院', '体調悪い', 'お腹痛い', '頭痛がする', '人間ドック受けたい', '風邪ひいた'],
 'ガゾリンスタンド': ['ガゾリンスタンド', '車が止まりそう', 'タイヤがパンクした', 'ガス欠', 'ワイパー交換', 'オイルが古い']}

上記では人間にわかりやすくデータを整えました。今度はモデルに渡す形式に整えます。

xは学習する文章、yはその答えとなるidで、最終的にone hot表現に変換します。

In [4]:
x_train = []
y_train = []
labels = []

In [5]:
for key, sentences in datasets.items():
    labels.append(key)
    label_index = labels.index(key)
    for sentence in sentences:
        x_train.append(sentence)
        y_train.append(label_index)

In [6]:
x_train[0:3], y_train[0:3]

(['ファッション', '帽子買いたい', 'ジーパン欲しい'], [0, 0, 0])

In [7]:
x_train[6:9], y_train[6:9]

(['スーパー', '夕食の食材買いたい', '野菜買わないと'], [1, 1, 1])

In [8]:
labels[0:3]

['ファッション', 'スーパー', 'レストラン']

In [9]:
train_data = list(zip(x_train, y_train))
shuffle(train_data)
x_train = [d[0] for d in train_data]
y_train = [d[1] for d in train_data]

In [10]:
x_train[0:3], y_train[0:3]

(['靴がボロくなった', '卵足りない', 'ワイパー交換'], [0, 1, 6])

In [11]:
x_train = np.array(x_train)
y_train = np_utils.to_categorical(np.array(y_train))

In [12]:
x_train[0:3], y_train[0:3]

(array(['靴がボロくなった', '卵足りない', 'ワイパー交換'], dtype='<U11'),
 array([[1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.]], dtype=float32))

次にモデルの保存先、tensorboardの保存先を指定します。

一度学習してモデルを保存している場合、以下のコメントアウトでロードできます。

ロードする際、tf.hubを利用するなどkeras外のオブジェクトは保存できない為、再度定義してcustom_objects指定する必要があります。

In [13]:
model = None
model_path = 'models/usex.h5'
log_path = 'logs/tboard'

In [14]:
# load model
# model = load_model(model_path, custom_objects={'USEXEmbeddingLayer': USEXEmbeddingLayer})

tf.hubについて

tf.hubはtensorflow_hubライブラリを使って[tfhub.dev](https://tfhub.dev/)から学習済みモデルを試す事ができるサービスです。

単語ベクトルを取り出す[word2vec](https://tfhub.dev/google/Wiki-words-500-with-normalization/1)や画像分類の[mobilenet](https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/classification/3)などを数行で試す事ができます。

転移学習用の学習済みモデルやFine-tuningにも利用できるモデルもあります。

今回は[universal-sentence-encoder-xling-many](https://tfhub.dev/google/universal-sentence-encoder-xling-many/1)を利用します。

まずは試してみましょう。

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

sentences = [
    "私は犬と一緒にビーチを散歩するのが好きです",
    "子犬は僕と一緒にビーチを歩くのが好きみたい",
    "ピザ食べたい",
]

g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  xling_8_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling-many/1")
  embedded_text = xling_8_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

session = tf.Session(graph=g)
session.run(init_op)

results = session.run(embedded_text, feed_dict={text_input: sentences})

session.close()

print(results)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0605 23:15:28.501245 140692289308416 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


[[ 0.00365919 -0.08720801  0.05245281 ... -0.05964206 -0.07168947
  -0.03018713]
 [-0.01025635 -0.07971002  0.05991658 ... -0.01768638 -0.04235772
  -0.00679488]
 [ 0.05342148 -0.09510113 -0.05290775 ... -0.08235259  0.02876618
  -0.07422873]]


それぞれの文章(sentence)をuniversal-sentence-encoder-xling-manyに渡し、文章ベクトルを受け取りました。

機械学習で分類するには文字もすべて数値型にする必要がありますが、この文章ベクトルが文章の意味を持った数値になります。

本当にこの数値が文章の意味を表しているのか少し実験してみます。

In [16]:
def cos_sim(a, b):
    return np.inner(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

similarity_matrix0and1 = cos_sim(results[0], results[1])
similarity_matrix1and2 = cos_sim(results[1], results[2])
similarity_matrix2and0 = cos_sim(results[2], results[0])

print(similarity_matrix0and1)
print(similarity_matrix1and2)
print(similarity_matrix2and0)

0.8589692
0.2273775
0.27653313


コサイン類似度を使ってベクトルの近さを測りました。

結果は0番目と1番目が文章として近く、2番目は0番目も1番目もかけ離れているようで、意味を持った数値だという事がわかります。

今回はtf.hubのuniversal-sentence-encoderを使って文章ベクトルを作りました。

__本来はmecab+gensim+rnnなどを利用して文章ベクトルを作ります__

__分類したい分野の(word2vecなどの教師なし学習に利用する)大量の文章を持っていれば自分で学習した方が効率がいいかもしれません。例えば自社で料理に関する文章を大量に持っている場合に、料理に関する自然言語分類をしたい時などです。__

__スモールデータの場合はtf.hubなどを使った転移学習は有効です。__

nnlm、word2vec、elmo、universal-sentence-encoderを試しましたが、私が持っている問題に関しては一番universal-sentence-encoderが優秀でした。

tf.hubを利用して転移学習するためのモデルの層を定義します。

ややこしく書いていますが、単に学習済みモデルを利用した文章ベクトルを取り出した結果を利用しています。(以下のようなLambdaで定義しても同様です)

```python
def create_embedding_lambda():
    embed = hub.Module(
        'https://tfhub.dev/google/universal-sentence-encoder-xling-many/1',
        trainable=False,
        name='USEXEmbeddingLayer_module',
    )
    def embedding(x):
        return embed(
            K.squeeze(K.cast(x, K.tf.string), axis=1),
            signature="default",
            as_dict=True,
        )["default"]
    return embedding

outputs = layers.Lambda(create_embedding_lambda(), output_shape=(512,))(input_text)
```

また、未検証ですが、tensorflow v2からは[tf.hub.KerasLayer](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer)に対応したモデルであればカスタムレイヤ作る必要がなくなります。

In [17]:
# define EmbeddingLayer
class USEXEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.name = 'USEXEmbeddingLayer'
        self.trainable = kwargs['trainable'] if 'trainable' in kwargs else False
        super(USEXEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.usex = hub.Module(
            'https://tfhub.dev/google/universal-sentence-encoder-xling-many/1',
            trainable=self.trainable,
            name="{}_module".format(self.name),
        )
        super(USEXEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.usex(
            K.squeeze(K.cast(x, K.tf.string), axis=1),
            as_dict=True,
            signature='default',
        )['default']
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 512)

作成したレイヤを用いてモデルを定義します。

独自レイヤを使っている以外は単純な全層結合だけです。

合間にランダムにノードを不活性化させるDropoutやバッチごとに正規化するBatchNormalizationを用いて過学習を抑制します。

In [18]:
# define model
inputs = layers.Input(shape=(1,), dtype='string')
outputs = USEXEmbeddingLayer()(inputs)
outputs = layers.Dense(512, activation='relu')(outputs)
outputs = layers.BatchNormalization()(outputs)
outputs = layers.Dropout(0.5)(outputs)
outputs = layers.Dense(512, activation='relu')(outputs)
outputs = layers.BatchNormalization()(outputs)
outputs = layers.Dropout(0.5)(outputs)
outputs = layers.Dense(len(labels), activation='softmax')(outputs)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0605 23:15:50.582492 140692289308416 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [19]:
model = Model(inputs=[inputs], outputs=outputs)
model.compile(
    optimizer=optimizers.rmsprop(
        lr=0.001,
        rho=0.9,
        epsilon=None,
        decay=0.0,
    ),
    loss='categorical_crossentropy',
    metrics=['acc'],
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
usex_embedding_layer_1 (USEX (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
__________

実際にトレーニングしてみましょう。

過学習し始めた際に学習を止めるEarlyStoppingや学習率を学習中に変更してくれるReduceLROnPlateauなどをcallbackに使ってます。

In [20]:
# train
model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=1024,
    validation_split=0.1,
    shuffle=True,
    callbacks=[
        EarlyStopping(
            monitor='val_acc',
            patience=1,
        ),
        ModelCheckpoint(
            filepath=model_path,
            monitor='val_loss',
            save_best_only=True,
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=1,
        ),
        TensorBoard(
            log_dir=log_path,
            write_graph=True,
        )
    ],
)

Train on 37 samples, validate on 5 samples
Epoch 1/100
37/37 [==============================] - 4s 113ms/step - loss: 3.2915 - acc: 0.0270 - val_loss: 1.1783 - val_acc: 0.8000
Epoch 2/100
37/37 [==============================] - 0s 3ms/step - loss: 0.8989 - acc: 0.7027 - val_loss: 1.1652 - val_acc: 0.8000


学習を終えたのでためしてみましょう。

In [21]:
# predict
sentence = 'ディナーどこにしよう'

results = model.predict(np.array([sentence]))

In [22]:
results

array([[1.4555707e-02, 1.2063762e-02, 9.2107445e-01, 1.8246198e-02,
        2.6356930e-02, 6.8286220e-03, 8.7419694e-04]], dtype=float32)

In [23]:
result = results[0]
indexes = list(range(len(labels)))
predictions = dict(zip(indexes, result))
predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
predictions = predictions[0:3]
for prediction in predictions:
    label = labels[prediction[0]]
    score = '{:.2%}'.format(prediction[1])
    print('{}:{}'.format(score, label))

92.11%:レストラン
2.64%:トイレ
1.82%:コンビニ


In [24]:
def generate_predicter(model, labels):
    indexes = list(range(len(labels)))

    def predicter(sentences):
        results = model.predict(np.array(sentences))
        for sentence_index, result in enumerate(results):
            sentence = sentences[sentence_index]
            print('====================')
            print('q: {}'.format(sentence))
            predictions = dict(zip(indexes, result))
            predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
            predictions = predictions[0:5]
            for prediction in predictions:
                index = prediction[0]
                label = labels[index]
                score = prediction[1]
                print('\n----------\nscore:{}\n{}'.format('{:.2%}'.format(score), label))

    return predicter

In [25]:
predicter = generate_predicter(model, labels)

In [26]:
predicter(['車が壊れた'])

q: 車が壊れた

----------
score:81.57%
ガゾリンスタンド

----------
score:11.88%
ファッション

----------
score:3.12%
病院

----------
score:1.25%
コンビニ

----------
score:1.03%
トイレ


API化する為にtensorflow servingに対応したprotocol buffers形式で保存し、serving.ipynbにうつりましょう。

In [27]:
# save pb
serving_model_path = 'models/serving/1'
tf.saved_model.simple_save(
    K.get_session(),
    serving_model_path,
    inputs={'inputs': model.input},
    outputs={t.name: t for t in model.outputs},
)

Instructions for updating:
Pass your op to the equivalent parameter main_op instead.


W0605 23:16:29.572692 140692289308416 tf_logging.py:125] From /usr/local/lib/python3.6/site-packages/tensorflow/python/saved_model/simple_save.py:85: calling SavedModelBuilder.add_meta_graph_and_variables (from tensorflow.python.saved_model.builder_impl) with legacy_init_op is deprecated and will be removed in a future version.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.


INFO:tensorflow:Assets added to graph.


I0605 23:16:29.578189 140692289308416 tf_logging.py:115] Assets added to graph.


INFO:tensorflow:No assets to write.


I0605 23:16:29.581063 140692289308416 tf_logging.py:115] No assets to write.


INFO:tensorflow:SavedModel written to: models/serving/1/saved_model.pb


I0605 23:16:46.790414 140692289308416 tf_logging.py:115] SavedModel written to: models/serving/1/saved_model.pb
